Importamos las librerias necesarias

In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sn
import ast
import json
import nltk
from textblob import*

## We will carry out a standardized process for each dataset:

*  JSON Import: We begin by loading the data from the corresponding JSON file.

*  Exploratory Analysis: We conduct an initial analysis of the data to understand its structure and content.

*  Removal of Unnecessary Columns: We identify columns that won't contribute relevant information to our analysis and remove them from the dataset.

*  Format Adjustment: We ensure that the format of the remaining columns is suitable for our analysis. This may involve data type conversion or format normalization.

*  Detection and Handling of Duplicates and Null Values: We identify duplicate records and manage null values in the dataset.

*  This standardized process will enable us to efficiently prepare the data for subsequent analysis.

*  Transform each dataset for the optimal processing of each endpoint







# Reviews

In [87]:
# Import the JSON files

# List to store JSON dictionaries from each line
data_list = []

# JSON file path
file_path = 'Data/australian_user_reviews.json'

# Open the file and process each line
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Use ast.literal_eval to convert the line into a dictionary
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error in line: {line}")
            continue

# Create a DataFrame from the list of dictionaries
df_user_reviews = pd.DataFrame(data_list)


### Limpieza y Normalizacion

In [88]:
df_user_reviews = pd.json_normalize(df_user_reviews.to_dict('records'), 'reviews', ['user_id', 'user_url'])


### Agregacion de la columna de sentyment analysis

In [89]:

#use lib nltk with vader_lexicon and sentiment for transform reviews in score_reviews
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

#Function to assign values according to the scale
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # Return neutral if it is empty or NaN
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 2  # Good score
        elif compound_score <= -0.05:
            return 0  # Bad score
        else:
            return 1
    else:
        return 1  # Return neutral for non-string values


#Convert column 'review' to str
df_user_reviews['review'] = df_user_reviews['review'].astype(str)

#apply function get_sentiment_score to column 'review'
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(get_sentiment_score)

#Eliminar la columna review ya que con la de sentiment_score no la necesitamos
df_user_reviews = df_user_reviews.drop(columns=['review', 'funny', 'last_edited', 'helpful', 'user_url'], axis=1)



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\joaqu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [90]:

# Define a function to remove the word "Posted" from a string
def remove_posted_prefix(string):
    return string.replace("Posted", "")

# Apply the 'remove_posted_prefix' function to the 'posted' column to clean the data
df_user_reviews["posted"] = df_user_reviews["posted"].apply(remove_posted_prefix)

# Filter the DataFrame to keep only the rows containing a comma in the 'posted' column
df_user_reviews = df_user_reviews[df_user_reviews["posted"].str.contains(",")]

# Create a new 'year' column that extracts the year from the 'posted' column
df_user_reviews["year"] = df_user_reviews["posted"].str.split(", ").str[-1].str[:4]

# Remove the original 'posted' column as we have extracted the relevant information
df_user_reviews = df_user_reviews.drop(columns=['posted'], axis=1)



In [91]:
df_user_reviews['item_id'] = df_user_reviews['item_id'].astype(int)

## Nulos y Duplicados 

In [92]:

# Remove rows with missing values in the 'item_id' column
df_user_reviews = df_user_reviews.dropna(subset=['item_id'])

# Remove rows with missing values in the 'recommend' column
df_user_reviews = df_user_reviews.dropna(subset=['recommend'])


In [93]:
df_user_reviews.duplicated().sum()

688

In [94]:
df_user_reviews = df_user_reviews.drop_duplicates()

In [95]:
df_user_reviews.isnull().sum()

item_id               0
recommend             0
user_id               0
sentiment_analysis    0
year                  0
dtype: int64

In [96]:
df_user_reviews

,item_id,recommend,user_id,sentiment_analysis,year
0,1250,True,76561197970982479,2,2011
1,22200,True,76561197970982479,2,2011
2,43110,True,76561197970982479,2,2011
3,251610,True,js41637,2,2014
4,227300,True,js41637,2,2013
...,...,...,...,...,...
59252,730,True,wayfeng,2,2015
59255,253980,True,76561198251004808,2,2015
59265,730,True,72947282842,0,2015
59267,730,True,ApxLGhost,2,2015


## Exportar el dataframe a parquet

In [97]:
df_user_reviews.to_parquet('Data/user_reviews.parquet')

# Items

In [98]:
# List to store JSON dictionaries from each line
data_list = []

# JSON file path
file_path = 'Data/australian_users_items.json'

# Open the file and process each line
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Use ast.literal_eval to convert the line into a dictionary
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error in line: {line}")
            continue

# Create a DataFrame from the list of dictionaries
df_user_items = pd.DataFrame(data_list)


## Limpieza y Normalizacion

In [99]:
# We use json_normalize to extract nested information from the 'items' column

df_user_items = pd.json_normalize(
    df_user_items.to_dict('records'),  # Convert the DataFrame into a list of dictionary-like records
    'items',                      # Name of the column with nested data
    ['user_id', 'items_count', 'steam_id', 'user_url']  # Additional columns to be retained
)


In [100]:
#We drop useless columns
df_user_items = df_user_items.drop(columns=['playtime_2weeks', 'item_name', 'items_count', 'steam_id', 'user_url'])

In [101]:
# List of columns you want to convert to the integer data type
columns_to_convert = ['item_id', 'playtime_forever']

# Iterate through the columns and convert them to integers, handling errors
for column in columns_to_convert:
    df_user_items[column] = pd.to_numeric(df_user_items[column], errors='coerce')

# Now, invalid values will be set as NaN in those columns




## Duplicados y Nulos

In [102]:
df_user_items.duplicated().sum()

59117

In [103]:
df_user_items = df_user_items.drop_duplicates()

In [104]:
df_user_items.isnull().sum()

item_id             0
playtime_forever    0
user_id             0
dtype: int64

In [105]:
df_user_items = df_user_items.dropna()

## Exportar el dataframe  parquet

In [106]:
df_user_items.to_parquet('Data/user_items.parquet')

## Games

In [107]:
# Read the file line by line and load each line as a JSON object
with open('Data\output_steam_games.json', 'r', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]

# Convert the list of JSON objects into a DataFrame
df_steam_games = pd.DataFrame(data)


## Limpieza y Normalizacion

In [108]:
df_steam_games = df_steam_games.rename(columns={'id': 'item_id'})




In [109]:
def convert_to_0(valor):
    try:
        return float(valor)
    except (ValueError, TypeError):
        try:
            # Intentar convertir a entero y luego a float con decimal 0
            return float(int(valor))
        except (ValueError, TypeError):
            return 0.0

df_steam_games['price'] = df_steam_games['price'].apply(convert_to_0)


In [110]:
# Extract the release year from the 'release_date' column and create a new 'release_year' column
df_steam_games['release_year'] = df_steam_games['release_date'].str.extract(r'(\d{4})')


In [111]:
# Drop the specified columns from the DataFrame
df_steam_games = df_steam_games.drop(['publisher', 'release_date', 'title', 'app_name', 'url', 'tags', 'reviews_url', 'specs', 'early_access'], axis=1)


In [112]:
# Lista de columnas a procesar
cols = ['genres']

# Itera a través de las columnas
for i in cols:
    # Convierte los valores de la columna en cadenas de texto
    df_steam_games[i] = df_steam_games[i].astype(str)
    
    # Elimina los corchetes '[' y ']' de los valores de la columna
    df_steam_games[i] = df_steam_games[i].str.replace('[', '').str.replace(']', '')

## Nulos y Duplicados

In [113]:
# Drop rows where all columns are null
df_steam_games = df_steam_games.dropna(how="all")

# Drop rows where the 'developer' field is NaN
df_steam_games = df_steam_games.dropna(subset=['developer'])

# Drop rows where the 'price' field is NaN
df_steam_games = df_steam_games.dropna(subset=['price'])

# Drop rows where the 'release_year' field is NaN
df_steam_games = df_steam_games.dropna(subset=['release_year'])

df_steam_games = df_steam_games.dropna(subset=['item_id'])


In [114]:
df_steam_games['item_id'] = df_steam_games['item_id'].astype(int)


In [115]:
df_steam_games.duplicated().value_counts()

False    27592
True         1
Name: count, dtype: int64

In [116]:
df_steam_games = df_steam_games.drop_duplicates()

##  Exportar el dataframe a parquet

In [117]:
df_steam_games.to_parquet('Data/steam_games.parquet')

# First Function

In [136]:
def developer(developer):
    df_developer = df_steam_games[df_steam_games["developer"] == developer]
    items_per_year = df_developer.groupby("release_year")["item_id"].count()

    # Filter the developer's DataFrame for free games (price zero):
    df_dev_free = df_developer[df_developer["price"] == 0]

    # Get the number of free items per year
    free_items = df_dev_free.groupby("release_year")["price"].count()  # Number of free items per year

    # Calculate the percentage of free content per year
    free_proportion = round((free_items / items_per_year) * 100, 2)

    # Rename the series to merge them into a DataFrame:
    items_per_year.name = "Number of Items"
    free_proportion.name = "Free Content"

    df1 = pd.merge(items_per_year, free_proportion, on="release_year").reset_index()
    df1 = df1.fillna(0)

    df1 = df1.rename(columns={"release_year": "Year"})

    # Format the Free Content column:
    df1["Free Content"] = df1["Free Content"].apply(lambda x: f"{x}%")

    # Convert the DataFrame to a dictionary
    dictionary = df1.to_dict(orient="records")
    del df_developer, items_per_year, df_dev_free, free_items, free_proportion, df1
    return dictionary



# Second Function

In [119]:
df_reviews_games = df_user_reviews.merge(df_steam_games[['item_id', 'price']])

In [120]:
def userdata(user_id):
    # Filter the data for the specified user
    user_data = df_reviews_games[df_reviews_games['user_id'] == user_id]

    # Calculate the amount of money spent by the user
    spent_money = user_data['price'].sum()

    # Calculate the recommendation percentage based on reviews.recommend
    recommendations = (user_data['recommend'] == True).sum()
    recommendation_percentage = recommendations / len(user_data) * 100

    # Calculate the number of items
    number_of_items = user_data['item_id'].nunique()

    # Create a dictionary with the results
    results = {
        'Amount of money spent': round(spent_money, 2),
        'Recommendation Percentage': recommendation_percentage,
        'Number of items': number_of_items
    }
    return results



In [121]:
userdata('Wackky')

{'Amount of money spent': 89.95,
 'Recommendation Percentage': 100.0,
 'Number of items': 5}

# Third Function

In [134]:
def UserForGenre(genre):
    
    # Filter games by the specified genre
    games_filtered = df_steam_games[df_steam_games['genres'].str.contains(genre, case=False, na=False)]

    # Merge df_user_items and df_games based on item_id and id
    merged_df = pd.merge(df_user_items, games_filtered, left_on='item_id', right_on='item_id', how='inner')

    # Group by year and user, calculate the playtime by user per year
    grouped = merged_df.groupby(['release_year', 'user_id'])['playtime_forever'].sum().reset_index()

    if len(grouped) > 0:
        # Find the user with the most playtime for the given genre
        max_user = grouped[grouped['playtime_forever'] == grouped.groupby('release_year')['playtime_forever'].transform('max')]['user_id'].values[0]
    else:
        return {"error": "No data available for the specified genre"}

    # Filter the data for the user with the most playtime
    user_data = grouped[grouped['user_id'] == max_user]

    # Remove years with 0 playtime
    user_data = user_data[user_data['playtime_forever'] > 0]

    # Sort years in descending order
    user_data = user_data.sort_values(by='release_year', ascending=False)

    # Convert playtime to integers
    user_data['playtime_forever'] = user_data['playtime_forever'].astype(int)

    # Create a list of accumulated playtime by year
    hours_by_year = [{'Year': int(year), 'Hours': int(hours)} for year, hours in zip(user_data['release_year'], user_data['playtime_forever'])]

    result = {
        "User with the Most Playtime for Genre " + genre: max_user,
        "Playtime": hours_by_year
    }

    return result


# Fourth Function

In [123]:
df_games_reviews = pd.merge(df_steam_games, df_user_reviews, how="inner", left_on="item_id", right_on="item_id")
df_games_reviews = df_games_reviews.drop(['price', 'item_id', 'release_year', 'user_id', 'item_id', 'sentiment_analysis'], axis=1)

In [124]:
def best_developer_year(year):
    df = df_games_reviews
    
    # Convert columns to the appropriate data types
    df["recommend"] = df["recommend"].astype(int)
    df["year"] = df["year"].astype(int)
    
    # Perform groupby and apply an aggregation function
    df = df.groupby(['developer', 'year'])['recommend'].sum().reset_index()

    # Rename the resulting column
    df = df.rename(columns={'recommend': 'total_true'})
    
    # Filter the data for the specified year
    filtered_df = df[df['year'] == year]
    if filtered_df.empty:
        return "No data found for that year"

    # Group by developer and sum recommended games
    developer_grouped = filtered_df.groupby('developer')['total_true'].sum().reset_index()

    # Sort in descending order
    developer_grouped = developer_grouped.sort_values(by='total_true', ascending=False)

    # Add the position rank
    developer_grouped['Position'] = range(1, len(developer_grouped) + 1)

    # Select the top 3 developers
    top_developers = developer_grouped.head(3)

    # Create the result in the desired format
    result = [{"Position {}: {}".format(row['Position'], row['developer']): row['total_true']} for i, row in top_developers.iterrows()]

    return result

# Fifth Function

In [125]:
df_games_reviews = pd.merge(df_steam_games, df_user_reviews, how="inner", left_on="item_id", right_on="item_id")

In [126]:
def developer_reviews_analysis(developer):
    df = df_games_reviews
    df = df.drop(['item_id', 'release_year', 'price'], axis= 1)
    # Filtramos las reseñas del desarrollador especificado
    df = df[df["developer"] == developer]

    # Contamos el número de reseñas positivas y negativas
    count_positive = df[df["sentiment_analysis"] == 2].shape[0]
    count_negative = df[df["sentiment_analysis"] == 0].shape[0]

    # Devolvemos el resultado
    return {
        "developer": developer,
        "negative": count_negative,
        "positive": count_positive,
    }
